All import

In [1]:
import librosa as lb
import librosa.display as display
import librosa.effects as effects
import numpy as np
import csv as csv
import os
import matplotlib.pyplot as plt
import pandas as pd
from IPython.display import Audio

from hmmlearn import hmm

import sys

defining pre_processing

In [2]:

def pre_processing(path):
  #load
  audio, sr = lb.load(path = path, mono = True)

  #pre-emp
  pre_emp_audio = effects.preemphasis(audio, 0.97)


  #removing silence in audio
  clips = effects.split(pre_emp_audio, top_db=25)
  silenced=[]
  for c in clips:
    data = audio[c[0]: c[1]]
    silenced.extend(data)

  silence = np.array(silenced)



  return silence, sr

EXAMPLE_PATH = "/home/axel1143/Scrivania/Voxceleb_wav/wav/id10261/1suWlhhvRcs/00011.wav"
BASE_TRAINING_PATH="/home/axel1143/Scrivania/Voxceleb_wav/wav/"
#BASE_PATH = "C:\\Users\\alexp\\Desktop\\Voxceleb_mat\\"
BASE_PATH = "/home/alex/Scrivania/Voxceleb_mat/"

**(example)** calculating mfcc, delta, delta2

In [ ]:
processed_audio, sr = pre_processing(EXAMPLE_PATH)
print(len(processed_audio)/sr)
mfcc = lb.feature.mfcc(y = processed_audio, sr = sr, n_mfcc = 13)
mfcc_delta = lb.feature.delta(mfcc)
mfcc_delta2 = lb.feature.delta(mfcc, order = 2)

Audio(processed_audio, rate = sr)



**(example)** show mfcc, delta & delta2

In [ ]:
fig, ax = plt.subplots(nrows =3, sharex =True)

img_mfcc = display.specshow(mfcc, ax=ax[0])

img_delta = display.specshow(mfcc_delta, ax=ax[1])

img_delta2 = display.specshow(mfcc_delta2, ax=ax[2])



**(example)** single execution example

In [ ]:
row_csv = []
row_csv.append("id11")
row_csv.append("pathblablabla")
print(row_csv)
for row in mfcc:
  row_csv_2 = []
  for value in row:
    row_csv_2.append(value)
  row_csv.append(row_csv_2)

for row in mfcc_delta:
  row_csv_2 = []
  for value in row:
    row_csv_2.append(value)
  row_csv.append(row_csv_2)

for row in mfcc_delta2:
  row_csv_2 = []
  for value in row:
    row_csv_2.append(value)
  row_csv.append(row_csv_2)

header = ["id","path",
          "mfcc1","mfcc2","mfcc3","mfcc4","mfcc5","mfcc6","mfcc7","mfcc8","mfcc9","mfcc10","mfcc11","mfcc12","mfcc13",
          "delta1","delta2","delta3","delta4","delta5","delta6","delta7","delta8","delta9","delta10","delta11","delta12","delta13",
          "delta2_1","delta2_2","delta2_3","delta2_4","delta2_5","delta2_6","delta2_7","delta2_8","delta2_9","delta2_10","delta2_11","delta2_12","delta2_13"]

with open("/content/drive/MyDrive/DATASET_VOXCELEB_1/file.csv", "w", encoding="UTF-8", newline="") as cs:
  writer = csv.writer(cs)
  writer.writerow(header)
  writer.writerow(row_csv)



**creating** csv


In [ ]:
header = ["id","path",
          "mfcc1","mfcc2","mfcc3","mfcc4","mfcc5","mfcc6","mfcc7","mfcc8","mfcc9","mfcc10","mfcc11","mfcc12","mfcc13",
          "delta1","delta2","delta3","delta4","delta5","delta6","delta7","delta8","delta9","delta10","delta11","delta12","delta13", 
          "delta2_1","delta2_2","delta2_3","delta2_4","delta2_5","delta2_6","delta2_7","delta2_8","delta2_9","delta2_10","delta2_11","delta2_12","delta2_13"]

with open(BASE_PATH + "id_mfcc_deltas_ridotto.csv", "w", encoding="UTF-8", newline="") as writefile:
  writer = csv.writer(writefile)
  writer.writerow(header)
  counter_speaker = 0
  for dir in os.listdir(BASE_TRAINING_PATH):
    if counter_speaker < 300:
        counter_speaker+=1
        count_audio = 0
        for dir1 in os.listdir(BASE_TRAINING_PATH+dir):
            if count_audio < 40:
                for file in os.listdir(BASE_TRAINING_PATH+dir+"/"+dir1):
                  if count_audio < 40:
                      if not os.path.isdir(BASE_TRAINING_PATH+dir+"/"+dir1+"/"+file):
                        count_audio +=1
                        COMPLETE_PATH = BASE_TRAINING_PATH+dir+"/"+dir1+"/"+file
                        processed_audio, sr = pre_processing(COMPLETE_PATH)

                        #mfcc = get_mfcc_coefficient(processed_audio, rs) #my mfcc
                        mfcc = lb.feature.mfcc(y = processed_audio, sr = sr, n_mfcc=13)
                        mfcc_delta = lb.feature.delta(mfcc)
                        mfcc_delta2 = lb.feature.delta(mfcc, order=2)

                        row_csv = []
                        row_csv.append(dir)
                        row_csv.append(COMPLETE_PATH)


                        for row in mfcc:
                          row_csv_2 = []
                          for value in row:
                            row_csv_2.append(value)
                          row_csv.append(row_csv_2)

                        for row in mfcc_delta:
                          row_csv_2 = []
                          for value in row:
                            row_csv_2.append(value)
                          row_csv.append(row_csv_2)

                        for row in mfcc_delta2:
                          row_csv_2 = []
                          for value in row:
                            row_csv_2.append(value)
                          row_csv.append(row_csv_2)

                        writer.writerow(row_csv)

        print(counter_speaker)
    else:
        continue

loading df

In [ ]:
df = pd.read_csv(BASE_PATH + "id_mfcc_deltas_ridotto.csv")

mfcc dataset con esplosione coefficienti

In [ ]:
features_mfcc = ["mfcc1", "mfcc2", "mfcc3", "mfcc4", "mfcc5", "mfcc6", "mfcc7", "mfcc8", "mfcc9", "mfcc10", "mfcc11",
            "mfcc12", "mfcc13"]

labels = df.loc[:, ["id"]].values #labael dataset
features_mfcc_values = df.loc[:, features_mfcc].values #mfcc dataset

del df

total = features_mfcc_values.shape[0] * features_mfcc_values.shape[1]
counter = 0
correct_dataset_mfcc = []
header = []

with open(BASE_PATH + "id_mfcc_solo_ridotto.csv", "w", encoding="UTF-8", newline="") as writefile:
    writer = csv.writer(writefile)
    for i in range(features_mfcc_values.shape[0]): #
        row_dataset = []
        #row_dataset.append(labels[i])
        for j in range(features_mfcc_values.shape[1]):
            features_mfcc_values[i][j] = features_mfcc_values[i][j].split()
            for k in range(len(features_mfcc_values[i][j])):
                features_mfcc_values[i][j][k] = features_mfcc_values[i][j][k].replace("[", "")
                features_mfcc_values[i][j][k] = features_mfcc_values[i][j][k].replace("]", "")
                features_mfcc_values[i][j][k] = features_mfcc_values[i][j][k].replace(",", "")
            for ele in features_mfcc_values[i][j]:
                if ele == "":
                    features_mfcc_values[i][j].remove(ele)
            for k in range(len(features_mfcc_values[i][j])):
                features_mfcc_values[i][j][k] = float(features_mfcc_values[i][j][k])
                row_dataset.append(features_mfcc_values[i][j][k])
        correct_dataset_mfcc.append(row_dataset)
        counter += 1
        print(counter, "/", features_mfcc_values.shape[0])

    len_max = 0
    for row in correct_dataset_mfcc:
        if len(row) > len_max:
            len_max = len(row)

    for i in range(len_max):
        header.append("mfcc_" + str(i))

    print(len_max)

    writer.writerow(header)
    writer.writerows(correct_dataset_mfcc)

del correct_dataset_mfcc
del features_mfcc_values

mfcc riempiti in maniera circolare

In [4]:
mfcc = pd.read_csv(BASE_PATH + "id_mfcc_solo_ridotto.csv")

mfcc_np = np.array(mfcc, dtype=np.float32)
index = 0
for row in mfcc_np:
    row_counter = 0
    for i in range(len(row)):
        if np.isnan(row[i]):
            row[i] = row[row_counter]
            row_counter+=1
    index += 1
    print(index)
print(mfcc_np)

pd.DataFrame(mfcc_np).to_csv(BASE_PATH + "mfcc_filled_circular.csv")

loaded csv
dataframe completed
          mfcc_0     mfcc_1     mfcc_2     mfcc_3      mfcc_4     mfcc_5  \
0     -257.97195 -230.95300 -224.49272 -223.02423 -217.985810 -200.15079   
1     -222.82971 -177.45796 -144.76396 -131.10254 -119.256035 -110.73129   
2     -256.69763 -230.69241 -231.23729 -239.27986 -243.750370 -222.89789   
3     -447.66022 -364.88420 -340.55670 -351.75708 -369.286770 -372.16720   
4     -345.65176 -337.67172 -342.77338 -333.55093 -330.994960 -328.49884   
...          ...        ...        ...        ...         ...        ...   
11995 -381.94952 -349.82830 -346.78622 -358.00244 -365.792800 -367.26624   
11996 -390.75390 -352.10690 -340.24660 -338.99420 -344.979800 -371.16300   
11997 -498.38303 -440.63090 -369.90802 -364.75470 -396.901600 -431.09670   
11998 -402.48422 -265.44583 -222.53365 -252.51994 -354.065500 -435.74020   
11999 -569.40330 -432.81006 -402.12128 -427.02840 -450.925200 -403.64014   

          mfcc_6     mfcc_7     mfcc_8     mfcc_9  ...  

vector quantization con kmeans

In [ ]:
from sklearn.cluster import KMeans

mfcc = pd.read_csv(BASE_PATH + "mfcc_filled_circular.csv")

mfcc_32 = np.array(mfcc, dtype= np.float32)
mfcc_32 = np.delete(mfcc_32, 0, axis=1)

X = mfcc_32.reshape((-1, 1))

print("reshaped complete")

kmeans = KMeans(n_clusters=50)
kmeans.fit(X)

values = kmeans.cluster_centers_.squeeze()
labels = kmeans.labels_

matrix = np.array(labels)
matrix = matrix.reshape(mfcc_32.shape)

pd.DataFrame(matrix).to_csv(BASE_PATH + "obs_for_hmm.csv")

test set & train set

In [6]:
observations = np.array(pd.read_csv(BASE_PATH + "obs_for_hmm.csv"))
observations = np.delete(observations, 0, axis=1)

index = 0
train_set = []
test_set = []
for row in observations:
    if 0 <= index <= 34:
        train_set.append(row)
        index+=1
    elif index <= 38:
        test_set.append(row)
        index+=1
    elif index == 39:
        test_set.append(row)
        index = 0

test_set = np.array(test_set)
train_set = np.array(train_set)


pd.DataFrame(train_set).to_csv(BASE_PATH + "train_set_hmm.csv")
pd.DataFrame(test_set).to_csv(BASE_PATH + "test_set_hmm.csv")

[[    0     0    26 ...    45    32    32]
 [    1    26    35 ...    14    22     1]
 [    2     0    26 ...     1     1    33]
 ...
 [11997    13    42 ...    36    21    36]
 [11998    49    37 ...    45     3    38]
 [11999    31    42 ...    22    33    33]]
1500
10500


In [3]:
train_set = np.array(pd.read_csv(BASE_PATH + "train_set_hmm.csv"))
train_set = np.delete(train_set, 0, axis=1)

test_set = np.array(pd.read_csv(BASE_PATH + "train_set_hmm.csv"))
test_set = np.delete(train_set, 0, axis=1)


In [5]:
index = 0
samples = []
model = []

for i in range(5):
    samples.append(train_set[index : index + 35])
    model.append(hmm.MultinomialHMM(n_components=50))
    model[i].fit(samples[i])
    print(model[i].transmat_)
    index += 35

    print("fitted" + str(index))


[[5.51469958e-02 3.22197460e-02 2.60996266e-03 ... 4.41496543e-02
  2.61474123e-03 7.37857827e-03]
 [3.33086523e-02 8.71013701e-02 2.71841022e-03 ... 4.65224636e-02
  3.19032764e-03 1.07341255e-02]
 [1.29312044e-03 1.28832494e-03 1.37204332e-01 ... 2.63817141e-03
  6.20351938e-02 1.34785823e-02]
 ...
 [7.73140157e-03 7.72760478e-03 9.41254410e-04 ... 7.63119656e-01
  4.75995890e-04 4.03347625e-04]
 [1.35646774e-03 1.55905244e-03 6.59418424e-02 ... 1.38989170e-03
  9.97157046e-02 1.61716692e-02]
 [6.71952049e-03 9.25452670e-03 2.50152332e-02 ... 2.04816173e-03
  2.83613461e-02 4.82023258e-02]]
fitted35
[[0.04848256 0.01771488 0.02688723 ... 0.0059994  0.05494801 0.03063855]
 [0.02146682 0.0540962  0.01372961 ... 0.00550881 0.02205571 0.01672486]
 [0.02511857 0.01086876 0.04971325 ... 0.01208002 0.04648296 0.03798508]
 ...
 [0.00787838 0.00547763 0.01634409 ... 0.08456297 0.01245924 0.01621827]
 [0.03809131 0.01298582 0.03426579 ... 0.00621844 0.11386389 0.03571769]
 [0.02846979 0.012434

In [57]:
tester = test_set[0:25]
correct_label = 0
counted = 0
corrected = 0



for i in range(tester.shape[0]):

    counted+=1

    if 0 <= i <= 4: correct_label = 0
    elif 5<= i <= 9: correct_label = 1
    elif 10<= i <= 14: correct_label = 2
    elif 15<= i <= 19: correct_label = 3
    elif 20<= i <= 24: correct_label = 4

    answers = []
    for mod in model:
        answers.append(mod.score([tester[i]]))

    answ = max(answers)

    pred_label = answers.index(answ)
    if pred_label == correct_label: corrected+=1

    #print(pred_label, " ", correct_label, " max:", answ, " score:", answers)

print(corrected/counted)

NotImplementedError: Must be overridden in subclass

In [62]:
mod = hmm.MultinomialHMM(n_components=50, algorithm=)

viterbi
